# Public Config

fuse now supports to run simulations with a public configuration file resembling the ones used for simulations within the XENONnT collaboration. Where ever possible we set the configuration values to values published in XENON papers. For resources like e.g. maps we use dummy inputs with fixed values. Simulations with the public configuration file will not be able to fully simulate the XENONnT detector but they can be used to get a starting point for simulations of a similar detector or to work on reconstruction algorithms based on data similar to XENONnT data.

## Imports and Data Preparation

In [ ]:
import fuse
import pandas as pd
import numpy as np

Similar to example 3_csv_input we will run a small simulation with mono energetic gammas. First we create a csv file listing the energy deposits of the gammas.  

In [ ]:
def monoenergetic_source(n, energy):  # number of photons, energy of photons
    df = pd.DataFrame()

    # radius and depth are in mm for microphysics instructions
    r = np.sqrt(np.random.uniform(0, 435600, n))  # 435600 = radius of TPC in mm^2
    theta = np.random.uniform(-np.pi, np.pi, n)
    df["xp"] = r * np.cos(theta)
    df["yp"] = r * np.sin(theta)
    df["zp"] = np.random.uniform(-1500, 0, n)  # 0 is the position of the gate electrode

    df["xp_pri"] = df["xp"]
    df["yp_pri"] = df["yp"]
    df["zp_pri"] = df["zp"]

    df["ed"] = np.array([energy] * n)
    # if the times are all zero they are distributed according to source_rate by fuse
    # custom interaction times in nanoseconds can also be used e.g np.arange(n)*1e9
    df["time"] = np.zeros(n)
    df["eventid"] = np.arange(n)

    df["type"] = np.repeat("gamma", n)

    df["trackid"] = np.ones(n)
    df["parentid"] = np.zeros(n, dtype=np.int32)
    df["creaproc"] = np.repeat("None", n)
    df["parenttype"] = np.repeat("None", n)
    df["edproc"] = np.repeat("None", n)

    return df

In [ ]:
microphysics_instructions = monoenergetic_source(100, 200)
microphysics_instructions.to_csv("monoenergetic_200keV.csv")

## Simulation

Now that we have some input data, we can set up the simulation config. This time we will use the new public_config_context. Resources for the simulation can be found in the files folder. Feel free to run simulations with your own configuration arguments and resources to see how your detector would react to the energy deposits!

In [ ]:
st = fuse.context.public_config_context(output_folder="./fuse_data")

st.set_config(
    {
        "path": ".",
        "file_name": "monoenergetic_200keV.csv",
        "n_interactions_per_chunk": 250,
    }
)

run_number = "00000"

### Running the Simulation

First lets run the microphysics simulation. For more details please take a look at the corresponding example notebooks.

In [ ]:
st.make("00000", "microphysics_summary")

In [ ]:
ms = st.get_df("00000", "microphysics_summary")
ms.head()

Next we run the detector simulation up to raw_records.

In [ ]:
st.make("00000", "raw_records")
rr = st.get_array("00000", "raw_records")

In [ ]:
import matplotlib.pyplot as plt

single_rr = rr[0]


x = np.arange(110) * single_rr["dt"]

plt.plot(x, single_rr["data"], color="black", label="Records Data")
plt.xlabel("Time (ns)")
plt.ylabel("ADC")
plt.legend()

plt.show()